In [ ]:
import pandas as pd
import numpy as np
import os
import math

"""
Iris classification based on provided data
Author: Paweł Kozikowski
"""

NEURAL_NETWORK_DATABASE: str = 'weights.npz'
IRIS_DATABASE: str = 'Iris.csv'

def sigmoid(value):
      return ((1 / (1 + (np.exp(-value)))))
def sigmoid_derivative(value):
      return ((np.exp(-value) / ((1 + np.exp(-value)) ** 2)))

def read_csv_data(csv_file: str) -> list[dict]:
      df = pd.read_csv(os.path.abspath(csv_file))
      df_format_to_dict: list[dict] = df.to_dict(orient="records")
      
      return df_format_to_dict

def format_csv_data(csv_data: dict) -> list[tuple[np.ndarray, np.ndarray]]:
      formatted_data_array: list[tuple[np.ndarray, np.ndarray]] = []
      
      for data_pack in csv_data:
            input_matrix: np.ndarray = np.array([
                  np.float64(data_pack["SepalLengthCm"]),
                  np.float64(data_pack["SepalWidthCm"]),
                  np.float64(data_pack["PetalLengthCm"]),
                  np.float64(data_pack["PetalWidthCm"])
            ])
            
            label_matrix: np.ndarray = np.array([])

            if (data_pack["Species"] == "Iris-setosa"):
                  label_matrix = np.array([1, 0, 0])
            elif (data_pack["Species"] == "Iris-versicolor"):
                  label_matrix = np.array([0, 1, 0])
            elif (data_pack["Species"] == "Iris-virginica"):
                  label_matrix = np.array([0, 0, 1])

            tupled_data: tuple[np.ndarray, np.ndarray] = (input_matrix, label_matrix)
            formatted_data_array.append(tupled_data)
      
      return formatted_data_array

def save_neural_network_parameters(npz_file: str, parameters: tuple[np.ndarray, np.ndarray]) -> None:
      """
      save_neural_network_parameters FUNCTION

      Parameters:
            npz_file: NPZ file where neural network`s data such as weights and biases will be stored
            parameters: Neural network`s data such as weights and biases that will be stored
      
      Returns:
            None

      Purpose:
            Saving newly calculated data about out neural network such as weights and biases to proper NPZ file
      """
      # Separation data on weights and biases
      new_weights: np.ndarray = parameters[0]
      new_biases: np.ndarray = parameters[1]
      
      # Saving data to provided NPZ file
      np.savez(npz_file, 
               weights=new_weights,
               biases=new_biases)
      
      return None

def load_neural_network_parameters(npz_file: str) -> tuple[np.ndarray, np.ndarray]:
      """
      load_neural_network_parameters FUNCTION

      Parameters:
            npz_file: NPZ file containg data about current weights and biases

      Returns:
            loaded_weights, loaded_biases
            Tuple containg current weights and biases of neural network
      
      Purpose:
            Loads data from proper database to use it in other algorithms
      """
      # Read data from NPZ file
      loaded_neural_network_data = np.load(npz_file)

      # Loads weights and biases from this file
      loaded_weights: np.ndarray = loaded_neural_network_data["weights"]
      loaded_biases: np.ndarray = loaded_neural_network_data["biases"]

      return loaded_weights, loaded_biases

def front_propagation(input_matrix: np.ndarray, weights_matrix: np.ndarray, biases_matrix: np.ndarray) -> np.ndarray:
      """
      front_propagation FUNCTION

      Parameters:
            input_matrix: Matrix containing input values for our neural network
            weights_matrix: Matrix containing weights values of our neural network
            biases_matrix: Matrix containing biases values for our neural network

      Returns:
            linear_output: Matrix containing pre activation sums on each neuron in our output layer

      Purpose:
            Front propagation algorithm calculate pre activation sum on each neuron in our output layer
            This sum after transforming by activation function can be readed as value representing which answer is most likely to be true
      """
      # Calculating Matrix containing pre activation sums on each neuron in output layer
      linear_output = input_matrix @ weights_matrix + biases_matrix

      return linear_output

def back_propagation(input_matrix: np.ndarray, sums_matrix: np.ndarray, target_matrix: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
      """
      back_propagation FUNCTION

      Parameters:
            input_matrix: Matrix containing input values for our neural network
            sums_matrix: Matrix contating pre activation values for neurons in our output layer
            target_matrix: Matrix containg prepared target values informing which answer is correct

      Returns:
            gradient_weights_matrix, gradient_bias_matrix
            Tuple containing gradients of weights and biases that are needed to be applied to current weights and biases
            in order to make our neural network "more intelligent"

      Purpose:
            Back propagation algorithm enables our neural network to be more intelligent after each contact with data
            By this algorithm weights and biases in our neural network are being more and more appropriate to inputs we deliver
            to input layer of our neural network
      """
      # Calculate post activation outputs of our neural network
      sigmoid_matrix: np.ndarray = sigmoid(sums_matrix)
      
      # Calculating final output value for output layer
      error_signal = 2 * (sigmoid_matrix - target_matrix)
      sigmoid_derivative_matrix = sigmoid_matrix * (1 - sigmoid_matrix) # Optimized version for calculating sigmoid derivative
      delta_layer = error_signal * sigmoid_derivative_matrix
      
      # Calculating weights gradient with use of outer multiplication
      gradient_weights_matrix: np.ndarray = np.outer(input_matrix, delta_layer)
      
      # Calculating biases gradient (delta_layer can be assigned)
      gradient_bias_matrix: np.ndarray = delta_layer

      return gradient_weights_matrix, gradient_bias_matrix
      
def single_train(input_matrix: np.ndarray, target_matrix: np.ndarray, learning_rate: np.float64 = 0.1) -> None:
      """
      single_train FUNCTION

      Parameters:
            input_matrix: Matrix containg input values for our neural network
            target_matrix: Matrix containg prepared target values informing which answer is correct
            learning_rate: Rate with which our neural network learns (Can be interpreted as jump in looking for minimum in our cost function)

      Returns:
            None

      Purpose:
            Adjust weights and biases of our Neural Network to perform better at presenting us correct answers
      """ 
      # Load current weights and biases of our neural network
      weights_matrix, biases_matrix = load_neural_network_parameters(NEURAL_NETWORK_DATABASE)
      # Front propagation
      sums_matrix: np.ndarray = front_propagation(input_matrix, weights_matrix, biases_matrix)
      
      # Back propagation
      weights_gradient, biases_gradient = back_propagation(input_matrix, sums_matrix, target_matrix)
      # Calculating new weights and biases for our neural network
      new_weights_matrix: np.ndarray = weights_matrix - learning_rate * weights_gradient
      new_biases_matrix: np.ndarray = biases_matrix - learning_rate * biases_gradient

      # Saving new data in database
      save_neural_network_parameters(NEURAL_NETWORK_DATABASE, (new_weights_matrix, new_biases_matrix))

      return None

def init_parameters(input_size: int, output_size: int) -> None:
      """
      init_paramaters FUNCTION

      Parameters:
            input_size: Size of input layer in our neural network
            output_size: Size of output layer in our neural network

      Returns:
            None
      
      Purpose:
            Our Neural Network need to be initialized with random weights and biases to actually change them and learn with each portion of data
      """
      # Initializing random weights of our neural network
      initialized_weights = np.random.rand(input_size, output_size)
      # Initializing random biases of our neural network
      initialized_biases = np.zeros(output_size)

      # Saving new weights and biases to provided database
      save_neural_network_parameters(NEURAL_NETWORK_DATABASE, (initialized_weights, initialized_biases))

      return None

def train_network(epochs: int, learning_rate: np.float64 = 0.1) -> None:
      """
      train_network FUNCTION

      Paramaters:
            epochs: Number of epchos user provide for our neural network
            What is epochs ? 
            Epochs is one training serie for our neural network in which all provieded data is propagated through our neural network
      
      Returns:
            None

      Purpose:
            This function enables our neural network to work and learn on big database.
            Each epchos makes our neural network "smarter" by adjusting each weight and bias
      """
      # Reading CSV data from Iris Database 
      iris_csv_data: list[dict] = read_csv_data(IRIS_DATABASE)
      # Formtting iris_csv_data
      formatted_iris_database: list[tuple[np.ndarray, np.ndarray]] = format_csv_data(iris_csv_data)

      # Loop which trains 
      for epoch in range(epochs):
            for input_matrix, target_matrix in formatted_iris_database:
                  # Do single training session
                  single_train(input_matrix, target_matrix, learning_rate)            

      return None

def predict(input_matrix: np.ndarray) -> tuple[str, np.ndarray]:
      """
      predict FUNCTION

      Paramters:
            input_matrix: Matrix containing inputs values for our neural network
      
      Returns:
            Name of predicted Iris type and matrix containg values of each neuron output
      
      Purpose:
            This function make predictions from inputs provided by user.
            Inputs matrix contains data about Iris characteristic such as
            1. Sepal Length 
            2. Sepal Width
            3. Petal Length
            4. Petal Width
      """
      # Loading current weights and biases of our neural network
      weights, biases = load_neural_network_parameters(NEURAL_NETWORK_DATABASE)
      # Calculating pre activation sums on each neuron based on loaded weight biases and provided input matrix
      sums = front_propagation(input_matrix, weights, biases)
      # Calculating post activation sums on each neuron
      outputs = sigmoid(sums)

      # Choosing index of matrix with largest value
      predicted_class_index = np.argmax(outputs)
      # Mapping answers for model to provide
      species: list[str] = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"] 

      return species[predicted_class_index], outputs


Random weights shape: (4, 3)
[[0.19345059 0.60766925 0.20824328]
 [0.31627351 0.79991851 0.54929838]
 [0.26063442 0.13703967 0.34359184]
 [0.22606951 0.9341868  0.21422426]]
Random inputs shape: (4,)
Random inputs shape transponse (4,)
[0.33027158 0.05187004 0.43370551 0.46907143]
Random biases shape: (3,)
[0.56986068 0.40977493 0.70589063]
Front propagation results: 
[0.86923837 1.14959782 1.05266375]
